<a href="https://colab.research.google.com/github/HwangHanJae/NLP_tutorial/blob/main/Dacon_NLP_tutorial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectorization 이란?

- NLP를 컴퓨터가 이해할 수 있게 수치로 바꾸는 것을 말합니다. 이때 벡터로 변환된 고유의 토큰들이 모인 집합을 vocabulary라 하며 vocabulary가 크면 클수록 학습이 오래 걸리게 됩니다. 다양한 벡터화 함수들이 존재하지만 이번 노트북에서는 가장 기본적으로 많이 쓰이는 세 가지를 소개하겠습니다.

- 토큰들을 벡터화하기 이전에 형태소 분석기를 사용하여 문자을 원하는 토큰들로 분리합니다.


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.6 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 448 kB 43.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [10]:
from konlpy.tag import Okt
import re
Okt = Okt()

sentences = ['자연어 처리는 정말 즐거워.', '즐거운 자연어 처리 다같이 해보자.']

tokens = []

for sentence in sentences:
  sentence = re.sub('[^가-힣a-z]','',sentence) # 간단한 전처리
  token = (Okt.morphs(sentence)) #형태소 분석기를 이용한 토큰 나누기
  tokens.append(' '.join(token))

print("형태소 단위로 문장 분리")
print("-----------------------")
print(tokens)

형태소 단위로 문장 분리
-----------------------
['자연어 처리 는 정말 즐거워', '즐거운 자연어 처리 다 같이 해보자']


# 1.One Hot Encoding
- 해당 단어가 존재하면 1, 그렇지 않으면 모두 0으로 표시되는 기법입니다.
- keras를 이용하면 쉽게 구현이 가능하며 texts_to_sequences를 활용해 각 토큰에 고유한 정수를 부여한 후 to_categorical을 활용하여 간단하게 구현합니다.

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

t = Tokenizer()
t.fit_on_texts(tokens)

print("각 토큰에게 고유의 정수 부여")
print("-"*20)
print(t.word_index)
print()

s1 = t.texts_to_sequences(tokens)[0]
print("부여된 정수로 표시된 문장1")
print("-"*20)
print(s1)
print()

s2 = t.texts_to_sequences(tokens)[1]
print("부여된 정수로 표시된 문장2")
print("-"*20)
print(s2)
print()

s1_one_hot = to_categorical(s1)
print("문장1의 ont-hot-encoding")
print("-"*20)
print(s1_one_hot)
print()

s2_one_hot = to_categorical(s2)
print("문장2의 one-hot-encoding")
print("-"*20)
print(s2_one_hot)

각 토큰에게 고유의 정수 부여
--------------------
{'자연어': 1, '처리': 2, '는': 3, '정말': 4, '즐거워': 5, '즐거운': 6, '다': 7, '같이': 8, '해보자': 9}

부여된 정수로 표시된 문장1
--------------------
[1, 2, 3, 4, 5]

부여된 정수로 표시된 문장2
--------------------
[6, 1, 2, 7, 8, 9]

문장1의 ont-hot-encoding
--------------------
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

문장2의 one-hot-encoding
--------------------
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


- 이 방식은 vocabulary의 크기가 커짐에 따라 많은 공간을 차지하게 되고 벡터가 굉장히 sparse 해지기 때문에 모델에게 좋은 특성을 알려주지 못하는 경우가 대부분입니다.

# 2.Count vectorization
- 이 방식은 vocabulary를 활용하여 각 문장이 갖고 있는 토큰의 count를 기반으로 문장을 vectorization 해줍니다. 아래 예시를 통해 알아봅시다.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

vectors = vectorizer.fit_transform(tokens) #여러 개의 문장을 넣어줘야 작동함

print(vectorizer.get_feature_names())
print(vectors.toarray())

['같이', '자연어', '정말', '즐거운', '즐거워', '처리', '해보자']
[[0 1 1 0 1 1 0]
 [1 1 0 1 0 1 1]]


벡터화 결과 각 문장은 vocabulary의 인덱스를 기준으로 카운트가 정수로 표시된 것을 알 수 있습니다.

또한, 이 부분에서 형태소 분석기의 중요성에 대해서 알 수 있습니다. 즐거운과 즐거워는 같은 의미를 갖는 토큰이지만 okt는 이를 구분해주지 못해서 다른 토큰으로 분리가 되었습니다.
이는 모델에서 같은 의미의 토큰을 다르게 학습할 수 있음을 의미합니다.

# 3.TfIdf

- 단어가 몇번 등장 했는지에 대한 정보
- 어떤 단어가 언급된 문서의 수가 적다면 그 단어는 문서를 분류하는데 있어서 중요한 단어

즉 등장 횟수도 많고 문서 분별력 있는 단어들을 점수화하여 벡터화를 한 것이 TfIdf기법입니다.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df = 0)
tfidf_vectorizer = tfidf.fit_transform(tokens)

#tf-idf dictionary
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)
print(tfidf_vectorizer.toarray())


['같이', '자연어', '정말', '즐거운', '즐거워', '처리', '해보자']
[[0.         0.40993715 0.57615236 0.         0.57615236 0.40993715
  0.        ]
 [0.49922133 0.35520009 0.         0.49922133 0.         0.35520009
  0.49922133]]


Count Vectorization과 동일한 방법이지만 단어들에 대한 가중치가 다르게 표시됩니다.

# 4.Padding

- 여기서 우리는 한 가지 궁금증이 생기게 됩니다. 우리가 사용하던 모델들은 DataFrame형식의 row별 동일한 column수를 가지게되는데 
NLP에서는 row별(문장별) column(토큰의 개수)가 같지 않아도 되는건가?? 
가변 길이의 문장들을 입력으로 넣어도 되는건가?

- 가변 길이의 입력을 받는 모형들이 존재하지만 아쉽게도 기본적으로는 문자의 길이를 동일하게 맞춰주어야 합니다. 문장의 길이를 맞춰주기 위하여 부족한 길이 만큼 0을 채워넣게 되는 이것이 Padding입니다.